# CNN2ID基于CNN的网络攻击检测

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt

from tqdm import tqdm

import sys
sys.path.append('../..')

from logger import setup_logging
from models import MLP
from utils import (
    dataset,
    test,
    train,
    train_copy,
    utils,
    visualisation
)

In [16]:
LOG_CONFIG_PATH = os.path.join(os.path.abspath("../.."), "logger", "logger_config.json")
LOG_DIR   = os.path.join(os.path.abspath("../.."), "logs")
DATA_DIR  = os.path.join(os.path.abspath("../.."), "data")
IMAGE_DIR = os.path.join(os.path.abspath("../.."), "images")

check GPU is available?

In [17]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [18]:
# Get the datasets
train_data, val_data, test_data = dataset.get_dataset(data_path=DATA_DIR, balanced=True)

# How many instances have we got?
print('# instances in training set: ', len(train_data))
print('# instances in validation set: ', len(val_data))
print('# instances in testing set: ', len(test_data))

batch_size = 64

# Create the dataloaders - for training, validation and testing
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset=val_data, batch_size=batch_size, shuffle=True)
test_loader  = torch.utils.data.DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)

# instances in training set:  140000
# instances in validation set:  485146
# instances in testing set:  485145


In [19]:
X_train = train_data.features
#y_train = pd.DataFrame(train_data.labels)
X_val = val_data.features
#y_val = pd.DataFrame(val_data.labels)
X_test = test_data.features
#y_test = pd.DataFrame(test_data.labels)


In [20]:
y_train = train_data.labels
y_val = val_data.labels
y_test = test_data.labels

In [21]:
y_test = y_test.squeeze()
y_val = y_val.squeeze()

In [22]:
#reshape the data for CNN pytorch
X_train = X_train.to_numpy().reshape(len(X_train), 1, X_train.shape[1])
X_val = X_val.to_numpy().reshape(len(X_val), 1, X_val.shape[1])
X_test = X_test.to_numpy().reshape(len(X_test), 1, X_test.shape[1])
X_train.shape, X_test.shape

((140000, 1, 49), (485145, 1, 49))

In [23]:
X_train, y_train

(array([[[0.18218218, 0.42792793, 0.73973974, ..., 0.        ,
          0.        , 0.        ]],
 
        [[0.18218218, 0.5005005 , 0.8018018 , ..., 0.        ,
          0.        , 0.        ]],
 
        [[0.68818819, 0.74474474, 0.        , ..., 0.96613885,
          0.90740741, 0.91591592]],
 
        ...,
 
        [[0.80272778, 0.20820821, 0.57207207, ..., 0.        ,
          0.        , 0.        ]],
 
        [[0.82282282, 0.72722723, 0.        , ..., 0.        ,
          0.        , 0.        ]],
 
        [[0.90712836, 0.20820821, 0.57207207, ..., 0.        ,
          0.        , 0.        ]]]),
 623961     0
 393314     0
 624080     0
 1430553    0
 636972     0
           ..
 259700     1
 757698     1
 521547     1
 1447653    1
 94017      1
 Name: label, Length: 140000, dtype: int64)

In [24]:
#Create model
class CNN2ID(nn.Module):
    def __init__(self):
        super(CNN2ID, self).__init__()
        self.conv1 = nn.Conv1d(in_channels = 1, out_channels=64, kernel_size=5, padding=2)
        self.batchnorm1 = nn.BatchNorm1d(64)
        self.maxpool1 = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv1d(in_channels = 64, out_channels=64, kernel_size=5, padding=2)
        self.batchnorm2 = nn.BatchNorm1d(64)
        self.maxpool2 = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv1d(in_channels = 64, out_channels=64, kernel_size=5, padding=2)
        self.batchnorm3 = nn.BatchNorm1d(64)
        self.maxpool3 = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(7*64, 64)#应该是这里的问题
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 7)#64, 3
        
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.batchnorm1(x)
        x = self.maxpool1(x)
        
        x = self.relu(self.conv2(x))
        x = self.batchnorm2(x)
        x = self.maxpool2(x)
        
        x = self.relu(self.conv3(x))
        x = self.batchnorm3(x)
        x = self.maxpool3(x)
        
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.softmax(self.fc3(x))
        
        return x

In [25]:
input_data = torch.randn((32, 1, 49))
model = CNN2ID()
output = model(input_data)

In [26]:
input_data = torch.randn((32, 1, 49))
model = CNN2ID()

# 逐层计算形状变化
print("Input shape:", input_data.shape)

x = model.conv1(input_data)
print("Conv1 output shape:", x.shape)

x = model.batchnorm1(x)
print("BatchNorm1 output shape:", x.shape)

x = model.maxpool1(x)
print("MaxPool1 output shape:", x.shape)

x = model.conv2(x)
print("Conv2 output shape:", x.shape)

x = model.batchnorm2(x)
print("BatchNorm2 output shape:", x.shape)

x = model.maxpool2(x)
print("MaxPool2 output shape:", x.shape)

x = model.conv3(x)
print("Conv3 output shape:", x.shape)

x = model.batchnorm3(x)
print("BatchNorm3 output shape:", x.shape)

x = model.maxpool3(x)
print("MaxPool3 output shape:", x.shape)

x = model.flatten(x)
print("Flatten output shape:", x.shape)

x = model.fc1(x)
print("FC1 output shape:", x.shape)

x = model.fc2(x)
print("FC2 output shape:", x.shape)

x = model.fc3(x)
print("FC3 output shape:", x.shape)

output = model.softmax(x)
print("Output shape:", output.shape)

Input shape: torch.Size([32, 1, 49])
Conv1 output shape: torch.Size([32, 64, 49])
BatchNorm1 output shape: torch.Size([32, 64, 49])
MaxPool1 output shape: torch.Size([32, 64, 25])
Conv2 output shape: torch.Size([32, 64, 25])
BatchNorm2 output shape: torch.Size([32, 64, 25])
MaxPool2 output shape: torch.Size([32, 64, 13])
Conv3 output shape: torch.Size([32, 64, 13])
BatchNorm3 output shape: torch.Size([32, 64, 13])
MaxPool3 output shape: torch.Size([32, 64, 7])
Flatten output shape: torch.Size([32, 448])
FC1 output shape: torch.Size([32, 64])
FC2 output shape: torch.Size([32, 64])
FC3 output shape: torch.Size([32, 7])
Output shape: torch.Size([32, 7])


In [27]:
# 训练过程
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def train(model, criterion, optimizer, X_train, y_train, epochs, batch_size):
    model.train()  # 设置模型为训练模式
    num_samples = X_train.shape[0]
    #y_train = y_train.squeeze()
    #num_batches = num_samples // batch_size
    y_train = np.array(y_train)

    for batch in range(num_samples // batch_size):
        start_index = batch * batch_size
        end_index = (batch + 1) * batch_size
        inputs = torch.tensor(X_train[start_index:end_index], dtype=torch.float32)
        labels = torch.tensor(y_train[start_index:end_index], dtype=torch.long)

        optimizer.zero_grad()  # 梯度清零

        outputs = model(inputs)  # 前向传播
        loss = criterion(outputs, labels)  # 计算损失
        _, preds = torch.max(outputs, 1)
        acc = torch.sum(preds ==labels.data) / labels.size(0)
        loss.backward()  # 反向传播
        optimizer.step()  # 更新参数
            
        
        #print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss/num_batches:.4f}")
        return loss, acc
#定义验证函数
def validate(model, criterion, X_test, y_test, epochs, batch_size):
    model.eval()  # 设置模型为评估模式
    epoch_loss = 0.0
    epoch_acc = 0.0
    y_test = np.array(y_test)
    with torch.no_grad():
        for batch in range(X_test.shape[0] // batch_size):
            start_index = batch * batch_size
            end_index = (batch + 1) * batch_size
            inputs = torch.tensor(X_test[start_index:end_index], dtype=torch.float32)
            labels = torch.tensor(y_test[start_index:end_index], dtype=torch.long)

            outputs = model(inputs)  # 前向传播
            loss = criterion(outputs, labels)  # 计算损失
            _, preds = torch.max(outputs, 1)  # 预测结果
            acc = torch.sum(preds == labels.data) / labels.size(0)  # 计算准确率

            epoch_loss += loss.item() * inputs.size(0)
            epoch_acc += acc.item() * inputs.size(0)

    epoch_loss /= X_test.shape[0]
    epoch_acc /= X_test.shape[0]
    return epoch_loss, epoch_acc

In [28]:
#主函数
epochs = 80
batch_size = 128
learning_rate = 0.001

#定义模型    
model = CNN2ID()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
val_interval = 10#每10次
#训练
for epoch in range(epochs):
    #训练，调用train函数 X_train, y_train
    train_loss, train_acc = train(model, criterion, optimizer, X_train, y_train, epochs, batch_size)
    #验证，调用val函数 X_test, y_test
    if epoch%val_interval == 0:
        val_loss, val_acc = validate(model, criterion, X_test, y_test, epochs, batch_size)
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
    #输出结果
    print(f"Epoch {epoch+1}/{epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    #print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

Val Loss: 1.9398, Val Acc: 0.0010
Epoch 1/80
Train Loss: 1.9349, Train Acc: 0.0312
Epoch 2/80
Train Loss: 1.8649, Train Acc: 1.0000
Epoch 3/80
Train Loss: 1.7849, Train Acc: 1.0000
Epoch 4/80
Train Loss: 1.6839, Train Acc: 1.0000
Epoch 5/80
Train Loss: 1.5646, Train Acc: 1.0000
Epoch 6/80
Train Loss: 1.4416, Train Acc: 1.0000
Epoch 7/80
Train Loss: 1.3358, Train Acc: 1.0000
Epoch 8/80
Train Loss: 1.2590, Train Acc: 1.0000
Epoch 9/80
Train Loss: 1.2111, Train Acc: 1.0000
Epoch 10/80
Train Loss: 1.1858, Train Acc: 1.0000
Val Loss: 1.9187, Val Acc: 0.6035
Epoch 11/80
Train Loss: 1.1740, Train Acc: 1.0000
Epoch 12/80
Train Loss: 1.1690, Train Acc: 1.0000
Epoch 13/80
Train Loss: 1.1669, Train Acc: 1.0000
Epoch 14/80
Train Loss: 1.1661, Train Acc: 1.0000
Epoch 15/80
Train Loss: 1.1657, Train Acc: 1.0000
Epoch 16/80
Train Loss: 1.1656, Train Acc: 1.0000
Epoch 17/80
Train Loss: 1.1655, Train Acc: 1.0000
Epoch 18/80
Train Loss: 1.1655, Train Acc: 1.0000
Epoch 19/80
Train Loss: 1.1654, Train Acc

In [ ]:
def train(model, criterion, optimizer, X_train, y_train, epochs, batch_size):
    model.train()  # 设置模型为训练模式
    num_samples = X_train.shape[0]
    y_train = np.array(y_train)
    train_loss = 0.0
    train_acc = 0.0
    train_correct = 0
    num_batches = 0

    for batch in range(num_samples // batch_size):
        start_index = batch * batch_size
        end_index = (batch + 1) * batch_size
        inputs = torch.tensor(X_train[start_index:end_index], dtype=torch.float32)
        labels = torch.tensor(y_train[start_index:end_index], dtype=torch.long)

        optimizer.zero_grad()  # 梯度清零

        outputs = model(inputs)  # 前向传播
        loss = criterion(outputs, labels)  # 计算损失
        _, preds = torch.max(outputs, 1)
        #acc = torch.sum(preds == labels.data) / labels.size(0)

        loss.backward()  # 反向传播
        optimizer.step()  # 更新参数
        correct = (preds == labels).sum().item()
        train_correct += correct
        train_loss += loss.item()  # 累积损失
        #train_acc += acc.item()  # 累积准确率
        num_batches += 1

    # 计算平均损失和准确率
    train_loss /= num_batches
    train_acc = train_correct/num_samples

    return train_loss, train_acc